### Load the ENV file

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
load_dotenv('./../.env')

os.getenv('LANGSMITH_PROJECT')

'MyFirstLangchainAutomation'

### Load the LLM

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.runnables import chain
import re

llm_qwen = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.5,
    max_tokens=400
)

# Making remove_think_block as a Runnable in order to chain it
@chain
def remove_think_block(text) -> str:
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL | re.IGNORECASE)

In [3]:
%pip install langchain_community


Note: you may need to restart the kernel to use updated packages.


### Message History with ChatMessageHistory

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [9]:
prompt_template = ChatPromptTemplate.from_messages([
    ('system', "Be concise in the response"),
    ('human', "{prompt}"),
    ('placeholder', "{history}")
])

chain = prompt_template | llm_qwen | StrOutputParser() | remove_think_block

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

session_id = "Dhanesh"

history = RunnableWithMessageHistory(
    chain, 
    get_session_history, 
    input_messages_key='prompt', 
    output_messages_key=None
)

response_1 = history.invoke(
    {"prompt": "When did India become independant ?"},
    config={"configurable": {"session_id": session_id}}
)

response_2 = history.invoke(
    {"prompt" : "How about Vietnam ?"},
    config= {"configurable": {"session_id": session_id}}
)

response_3 = history.invoke(
    {"prompt" : "And Pakistan ?"},
    config= {"configurable": {"session_id": session_id}}
)

get_session_history(session_id).clear()

print(response_1)
print(response_2)
print(response_3)



India became independent on **August 15, 1947**.


Vietnam declared independence on **September 2, 1945**, during the August Revolution, led by Ho Chi Minh. Full international recognition followed later.


Pakistan became independent on **August 14, 1947**, alongside India, as part of the partition of British India. The date marks the end of British rule and the creation of two separate nations.


### Chat Message History with SQLChatMessageHistory

In [6]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

prompt_template = ChatPromptTemplate.from_messages([
    ('system', "Be concise in the response"),
    ('human', "{prompt}"),
    ('placeholder', "{history}")
])

chain = prompt_template | llm_qwen | StrOutputParser() | remove_think_block

def get_session_history(session_id: str):
    return SQLChatMessageHistory(session_id=session_id, connection="sqlite:///chathistory.db")

history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="prompt",
    output_messages_key=None
)

session_id = "Dhanesh"
get_session_history(session_id).clear()

response_1 = history.invoke(
    {"prompt": "When did India become independant ?"},
    config={"configurable": {"session_id": session_id}}
)

response_2 = history.invoke(
    {"prompt" : "How about Vietnam ?"},
    config= {"configurable": {"session_id": session_id}}
)

response_3 = history.invoke(
    {"prompt" : "And Pakistan ?"},
    config= {"configurable": {"session_id": session_id}}
)


print(response_1)
print(response_2)
print(response_3)



India became independent on August 15, 1947.


Vietnam declared independence on September 2, 1945, following the August Revolution. Full independence from France was formalized through the Geneva Accords in 1954, ending French colonial rule.


Pakistan gained independence from British rule on August 14, 1947, following the partition of British India.
